In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

22/03/02 16:16:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
pyspark.__version__

'3.0.3'

In [3]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv 

--2022-03-01 15:38:08--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.132.17
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.132.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  29.2MB/s    in 23s     

2022-03-01 15:38:32 (29.9 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [4]:
!wc -l fhvhv_tripdata_2021-02.csv

11613943 fhvhv_tripdata_2021-02.csv


In [7]:
!head -n 101 fhvhv_tripdata_2021-02.csv > 02_head.csv

In [10]:
import pandas as pd

In [9]:
df_fhvhv_pd = pd.read_csv('02_head.csv')

In [10]:
df_fhvhv_pd.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [12]:
spark.createDataFrame(df_fhvhv_pd).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [3]:
from pyspark.sql import types

In [4]:
schema = types.StructType([
    types.StructField('hvfhs_license_num',types.StringType(),True),
    types.StructField('dispatching_base_num',types.StringType(),True),
    types.StructField('pickup_datetime',types.TimestampType(),True),
    types.StructField('dropoff_datetime',types.TimestampType(),True),
    types.StructField('PULocationID',types.IntegerType(),True),
    types.StructField('DOLocationID',types.IntegerType(),True),
    types.StructField('SR_Flag',types.DoubleType(),True)])

In [5]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [3]:
df.repartition(24).write.parquet('fhvhv/2021/02/')

NameError: name 'df' is not defined

In [3]:
df = spark.read.parquet('fhvhv/2021/02/')

In [4]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-02-02 22:06:25|2021-02-02 22:13:17|         126|         147|   null|
|           HV0003|              B02836|2021-02-05 13:37:18|2021-02-05 13:52:28|          85|          72|   null|
|           HV0003|              B02872|2021-02-05 20:59:47|2021-02-05 21:19:44|         229|         261|   null|
|           HV0005|              B02510|2021-02-04 14:19:42|2021-02-04 14:49:20|         260|         160|   null|
|           HV0003|              B02682|2021-02-05 08:07:49|2021-02-05 09:00:13|         140|          72|   null|
|           HV0003|              B02764|2021-02-05 19:18:08|2021-02-05 19:31:58|

In [4]:
df.registerTempTable('fhvhv')

In [6]:
df_count_records = spark.sql("""
SELECT 
    COUNT(*)
FROM fhvhv
WHERE date_trunc('day',pickup_datetime) = '2021-02-15'

""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [7]:
df_trip_duration = spark.sql("""
SELECT 
        *,
        unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) AS trip_duration
FROM fhvhv
ORDER BY unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime) DESC

""").show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|trip_duration|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+-------------+
|           HV0005|              B02510|2021-02-11 13:40:44|2021-02-12 10:39:44|         247|          41|   null|        75540|
|           HV0004|              B02800|2021-02-17 15:54:53|2021-02-18 07:48:34|         242|         254|   null|        57221|
|           HV0004|              B02800|2021-02-20 12:08:15|2021-02-21 00:22:14|         188|          55|   null|        44039|
|           HV0003|              B02864|2021-02-03 20:24:25|2021-02-04 07:41:58|          51|         147|   null|        40653|
|           HV0003|              B02887|2021-02-19 23:17:44|2021-02-20 09:44:01|         210|    

In [8]:
df_trip_duration = spark.sql("""
SELECT dispatching_base_num, COUNT(*)
FROM fhvhv
GROUP BY dispatching_base_num
ORDER BY COUNT(*)
""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B03136|    1741|
|              B02844|    3502|
|              B02512|   41043|
|              B02865|   76160|
|              B02800|   84277|
|              B02870|  101945|
|              B02395|  112433|
|              B02880|  115716|
|              B02836|  128978|
|              B02889|  138762|
|              B02888|  169167|
|              B02835|  189031|
|              B02877|  198938|
|              B02867|  200530|
|              B02879|  210137|
|              B02876|  215693|
|              B02882|  232173|
|              B02884|  244963|
|              B02883|  251617|
|              B02617|  274510|
+--------------------+--------+
only showing top 20 rows



In [9]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2022-03-01 16:51:50--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.137.80
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.137.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv.2’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-03-01 16:51:50 (55.2 MB/s) - ‘taxi+_zone_lookup.csv.2’ saved [12322/12322]



In [5]:
df_lookup = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [8]:
df_lookup.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [6]:
df_lookup.registerTempTable('look_up')

In [9]:
df_trip_1 = spark.sql("""
with tab_1 as (SELECT COALESCE(p.zone, 'Unknown') as pickup_zone, COALESCE(d.zone, 'Unknown') as dropoff_zone
FROM fhvhv f
INNER JOIN look_up p
ON p.LocationID = f.PULocationID
INNER JOIN look_up d
ON p.LocationID = f.DOLocationID)
SELECT CONCAT_WS('/',pickup_zone,dropoff_zone ) pickup_dropoff, count(*)
FROM tab_1
group by 1
order by 2 desc
""").show()

+--------------------+--------+
|      pickup_dropoff|count(1)|
+--------------------+--------+
|East New York/Gov...|  135123|
|Borough Park/Gove...|  111987|
|East New York/Corona|   90082|
|Canarsie/Governor...|   84078|
|Crown Heights Nor...|   77928|
| Borough Park/Corona|   74658|
|     Canarsie/Corona|   56052|
|Bay Ridge/Governo...|   53802|
|Crown Heights Nor...|   51952|
|East New York/Fla...|   45041|
|East New York/Jam...|   45041|
|East New York/Red...|   45041|
|East New York/Cen...|   45041|
|East New York/Lon...|   45041|
|East New York/Rid...|   45041|
|East New York/DUM...|   45041|
|East New York/Gre...|   45041|
|East New York/Wil...|   45041|
|East New York/Spr...|   45041|
|East New York/Mid...|   45041|
+--------------------+--------+
only showing top 20 rows

